# Selective Missing Data Prediction for Memory-based Collaborative Filtering

*Submitted in partial fulfillment of the requirements for the degree of Master of Analytics, RMIT University, Melbourne*

*Report Submission Date: June 14, 2021*
***

<p style="text-align:center;">
    <img src="images/cf_schematic.png" alt="collaborative-filtering-schematic" title="Collaborative Filtering Method" width="400" height="400"><br><center><i>Fig. 1: Collaborative Filtering recommends items based on how similar users liked an item <a id="ref4" href=#4>[4]</a></i></center>
</p>

# Table of Contents

* [1 Introduction](#Introduction)
    * [1.1 Objective](#Objective)
    * [1.2 About Collaborative Filtering](#About-Collaborative-Filtering)
    * [1.3 Working Principle & Assumptions](#Working-Principle-&-Assumptions)
    * [1.4 Components](#Components)
    * [1.5 Similarity Computation Criteria](#Similarity-Computation-Criteria)
* [2 Main Problem: Data Sparsity](#Main-Problem:-Data-Sparsity)
* [3 Previous Proposed Solutions & Drawbacks](#Previous-Proposed-Solutions-&-Drawbacks)
* [4 Current Proposed Solution](#Current-Proposed-Solution)
    * [4.1 Working Principle of EMDP](#Working-Principle-of-EMDP)
    * [4.2 Similar Neighbors Selection by PCC](#Similar-Neighbors-Selection-by-PCC)
    * [4.3 Significance Weighting](#Significance-Weighting)
    * [4.4 Similarity Thresholds](#Similarity-Thresholds)
    * [4.5 Missing Data Prediction](#Missing-Data-Prediction)
    * [4.6 Prediction for Active Users](#Prediction-for-Active-Users)
* [5 Implementation](#Implementation)
    * [5.1 About MovieLens 100K Dataset](#About-MovieLens-100K-Dataset)
    * [5.2 Data Import](#Data-Import)
    * [5.3 Hold-Out Sampling](#Hold-Out-Sampling)
    * [5.4 Matrix Conversion](#Matrix-Conversion)
    * [5.5 Similarity Matrix Calculation](#Similarity-Matrix-Calculation)
    * [5.6 Predict Missing Values](#Predict-Missing-Values)
* [6 Evaluation](#Evaluation)
    * [6.1 PCC Calculation](#PCC-Calculation)
    * [6.2 Prediction on Test Set](#Prediction-on-Test-Set)
    * [6.3 MAE and RMSE Computation](#MAE-and-RMSE-Computation)
* [7 Impact of EMDP](#Impact-of-EMDP)
* [8 Ending Notes](#Ending-Notes)
* [9 References](#References)

## Introduction

### Objective

In Recommender Systems, k-nearest neighbor-based Collaborative Filtering (also called memory-based Collaborative Filtering) faces a common challenge of missing values, which data scientists and researchers have been working hard to solve and have proposed solutions in the published paper : _"Effective Missing Data Prediction for Collaborative Filtering"_ <a id="ref3" href=#3>[3]</a>.

Based on the solution proposed in the paper, in this report we discuss and implement a solution framework in Python for effective missing data prediction algorithm in Collaborative Filtering. This is performed without using any Recommender Systems-related libraries. Also, this report discusses how and why the solution provided is able to tackle the missing value problem.

### About Collaborative Filtering

Collaborative filtering is one of the most popular and successful techniques used by recommender systems which automatically predicts or filters the interests of an active user by collecting or collaborating information about preferences or tastes from other similar users or items. These kinds of techniques are widely employed in several large and popular commercial systems, such as Amazon, Ebay, Netflix, Facebook, Spotify etc.

<p style="text-align:center;">
    <img src="images/netflix_recsys.png" alt="netflix-recsys" title="Netflix Recommendation System" width="600"><br><center><i>Fig. 2: An example of a personalized recommendation on Netflix homepage <a id="ref6" href=#6>[6]</a></i></center>
</p>

In the above image, we can see an example of a personalized recommendation on Netflix homepage. Netflix’s own recommendation system (termed as _The Netflix Experience_) is powered by a family of ranking algorithms, each optimized for a different purpose. For instance, the _Top Picks_ row on the homepage makes recommendations based on a personalized ranking of videos, and the _Trending Now_ row also incorporates recent popularity trends. The _Because You Watched_ row shows recommendations of similar movie/series based on user's watching history. These algorithms, along with many others, are used together to construct personalized home pages for over 100 million members.

<p style="text-align:center;">
    <img src="images/netflix_workflow.png" alt="netflix-workflow" title="Netflix Recommendation Workflow" width="600"><br><center><i>Fig. 3: Netflix Recommendation Workflow <a id="ref7" href=#7>[7]</a></i></center>
</p>

### Working Principle & Assumptions

It builds on the idea of recommending an item to an active user depending on other like-minded users. The underlying assumption or hypothesis here is :

>If person $A$ has the same opinion as person $B$ on an issue, $A$ is more likely to also have $B$'s opinion on a different issue than that of a randomly chosen person.

In terms of user-item scenario, although users have their own rating style, in practice if an item is a very popular item and has obtained a very high average rating from other users, then the active user will have a high probability to give this item a good rating too.

<p style="text-align:center;">
    <img src="images/user_item_mat.png" alt="user-item-matrix" title="Matrix Factorization in Collaborative Filtering" width="600"><br><center><i>Fig. 4: Working of Collaborative Filtering Process: User-Item Matrix <a id="ref8" href=#8>[8]</a></i></center>
</p>

### Components

Collaborative filtering systems typically contains a user-rating matrix that includes:

- Set of users
- Set of items
- Set of opinions about items: ratings, reviews, purchases etc.

It can be classified into two types: _Memory-based_ and _Model-based_.

<p style="text-align:center;">
    <img src="images/cf_types.png" alt="cf-types" title="Types of Collaborative Filtering" width="700"><br><center><i>Fig. 5: Examples of (a) Memory-based Collaborative Filtering and (b) Model-based Collaborative Filtering <a id="ref2" href=#2>[2]</a></i></center>
</p>

_Memory-based methods_ include:
-	User-based approach, which predicts the ratings of an active user based on the ratings of similar users having similar rating styles.
-	Item-based approach, which predicts the ratings of an active user based on the ratings of similar items for each item rated by the user.

_Model-based methods_ include:
-	Clustering models
-	Aspect models
-	Latent factor models

<p style="text-align:center;">
    <img src="images/memory_based_types.png" alt="memory-based-types" title="Types of Memory-based Collaborative Filtering" width="700"><br><center><i>Fig. 6: Examples of (a) User-based approach and (b) Item-based approach <a id="ref5" href=#5>[5]</a></i></center>
</p>

However, model-based methods are generally time-consuming to build and update, and are unable to cover diverse users ranges like memory-based methods.

### Similarity Computation Criteria

Notable similarity computation algorithms include _Pearson Correlation Coefficient_ (PCC) and _Vector Space Similarity_ (VSS) algorithms. PCC-based collaborative filtering generally can achieve higher performance than the other popular algorithm VSS, since it considers the differences of user rating styles. Also, their implementation is easier.

## Main Problem: Data Sparsity

The main issue with collaborative filtering is **Data Sparsity**, i.e., insufficient information on a user's rating history.
User-item matrix of commercial recommendation system is generally very sparse and the density of available ratings is often less than 1%. This is because every user is practically able to rate a limited or only few items at the same time. This sparsity or missing data in the user-item matrix directly leads to the prediction inaccuracy in collaborative filtering, thereby producing inaccurate recommendation results.

## Previous Proposed Solutions & Drawbacks

Some of the previously proposed solution and their drawbacks are :

- _Cluster-based smoothing_: Introducing cluster-based smoothing methods to fill the missing values can improve the filtering performance. This clusters the users using K-means first, and then predicts all the missing data based on the ratings of Top-N most similar users in the similar clusters. But it has a few drawbacks as well :

 - The clustering algorithm limits the diversity of users in each cluster.

 - Clustering results of K-means relies on the preselected K users.
 
 - For an active user who hasn't rated enough items or doesn't have enough similar users, predicting all the missing data could bring negative influence for the recommendation.
 

- _Dimensionality reduction_:  Dimensionality reduction approaches have been proposed to handle the data sparsity issue, which delete any unrelated or insignificant users or items. But this could discard some important information of the user-item matrix.


## Current Proposed Solution

The current proposed solution to tackle the data sparsity issue and therefore, increase density of user-item matrix is called _Effective Missing Data Prediction (EMDP)_ algorithm.

### Working Principle of EMDP

Every missing data in the matrix is evaluated by utilizing the available information both from users and items (similar neighbors). Based on certain threshold values of confidence, missing data are selectively given prediction ratings. The thresholds decide whether missing data prediction will bring positive influence for the recommendation of active users :

- If yes, then missing values are predicted by weighting of user and item similarities.
- Otherwise, the missing data are set to zero. 

This consideration is different from all other existing methods of prediction which predict all missing data in the user-item matrix, giving rise to inaccurate predictions.

### Similar Neighbors Selection by PCC

The user and item similarities are calculated using Pearson Correlation Coefficient (PCC) as per following formulae :

1. Similarity between users $a$ and $u$ :

$$\boxed{\text{Sim}(a,u) = \frac{\sum_{i \in I(a) \cap I(u)}^{}{\left( r_{a,i} - {\overline{r}}_{a} \right) \cdot \left( r_{u,i} - {\overline{r}}_{u} \right)}}{\sqrt{\sum_{i \in I(a) \cap I(u)}^{}\left( r_{a,i} - {\overline{r}}_{a} \right)^{2}} \cdot \sqrt{\sum_{i \in I(a) \cap I(u)}^{}\left( r_{u,i} - {\overline{r}}_{u} \right)^{2}}}}$$

where,<br> 
$r_{a,i} \rightarrow$ rating user $a$ gave item $i$, and<br>
${\overline{r}}_{a} \rightarrow$ averaging rate of user $a$.

2. Similarity between items $i$ and $j$:

$$\boxed{\text{Sim}(i,j) = \frac{\sum_{u \in U(i) \cap U(j)}^{}{(r_{u,i} - {\overline{r}}_{i}) \cdot (r_{u,j} - {\overline{r}}_{j})}}{\sqrt{\sum_{u \in U(i) \cap U(j)}^{}\left( r_{u,i} - {\overline{r}}_{i} \right)^{2}} \cdot \sqrt{\sum_{u \in U(i) \cap U(j)}^{}\left( r_{u,j} - {\overline{r}}_{j} \right)^{2}}}}$$

where,<br>
$r_{u,i} \rightarrow$ rating user $u$ gave item $i$, and<br>
${\overline{r}}_{i} \rightarrow$ average rating of item $i$.

### Significance Weighting

Since PCC tends to overestimate the similarities of users who happen to have rated a few items identically, but may not have similar overall preferences, a correlation significance weighting factor $\gamma$ (gamma) is introduced. This factor would devalue similarity weights that were based on a small number of co-rated items.

$$\boxed{Sim'(a,u) = \frac{Min(|I_{a} \cap I_{u}\ |,\gamma)}{\gamma} \cdot Sim(a,u)}$$

where $|I_{a} \cap I_{u}\ | \rightarrow$ number of items which user $a$ and user $u$ rated in common.

Likewise, for similarity between items a weighting factor $\delta$ (delta) is introduced which would devalue similarity weights that were based on a small number of users who rated both items.

$$\boxed{Sim'(i,j) = \frac{Min(|U_{i} \cap U_{j}|,\delta)}{\delta} \cdot Sim(i,j)}$$

where $|U_{i} \cap U_{j}| \rightarrow$ number of users who rated both item $i$ and item $j$.

### Similarity Thresholds

Next, threshold values $\eta$ and $\theta$ are defined for user and item similarities, respectively. If the similarity between a user and its neighbor exceeds $\eta$ , then this neighboring user is selected as being similar. Likewise, if the similarity between an item and its neighbor exceeds $\theta$, then this neighboring item is selected as being similar. Therefore, for each missing data $r_{u,i}$ , two sets are generated:

$$\boxed{S(u) = \ \left\{ u_{a} \middle| Sim'(u_{a},\ u)\  > \ \eta,\ u_{a} \neq \ u \right\}}$$

where $u \rightarrow$ active user and $u_{a} \rightarrow$ similar user;

$$\boxed{S(i) = \ \left\{ i_{k} \middle| Sim'(i_{k},\ i)\  > \ \theta,\ i_{k} \neq \ i \right\}}$$

where $i \rightarrow$ item and $i_{k} \rightarrow$ similar item.

>__NOTE__ : Thresholds $\gamma$, $\delta$, $\eta$ and $\theta$ need to be fine-tuned as:
>- too high value causes shortage of similar users or items (highly sparse user-item matrix).
>- too low value brings too many similar users or items causing overestimation (highly dense user-item matrix).

### Missing Data Prediction

The user-based and item-based approaches are systematically combined in order to take advantage of user correlations and item correlations in the user-item matrix. This is done because in practice, predicting missing data only using user-based approaches or only using item-based approaches potentially ignores valuable information required to make more accurate prediction.

Based on the sets $S(u)$ and $S(i)$ generated for each missing data $r_{u,i}$ the following four combination cases are possible and their respective predictions $P\left( r_{u,i} \right)$ are given as :

Case I:
User $u$ has similar users as well as item $i$ has similar items $\Rightarrow S(u) \neq \varnothing \land S(i) \neq \varnothing$ :

$$\boxed{P\left( r_{u,i} \right) = \ \lambda \times \left( \overline{u}\  + \frac{\sum_{u_{a} \in S(u)}^{}{Sim'\left( u_{a},\ u \right) \cdot \left( r_{u_{a},i} - {\overline{u}}_{a} \right)}}{\sum_{u_{a} \in S(u)}^{}{Sim'\left( u_{a},\ u \right)}} \right) + (1 - \lambda) \times \left( \overline{i}\  + \frac{\sum_{i_{k} \in S(i)}^{}{Sim'\left( i_{k},\ i \right) \cdot \left( r_{u,i_{k}} - {\overline{i}}_{k} \right)}}{\sum_{i_{k} \in S(i)}^{}{Sim'\left( i_{k},\ i \right)}} \right)}$$

Case II:
User $u$ has similar users, but item $i$ doesn't have similar items $\Rightarrow S(u) \neq \varnothing \land S(i) = \varnothing$ :

$$\boxed{P\left( r_{u,i} \right) = \ \overline{u}\  + \frac{\sum_{u_{a} \in S(u)}^{}{Sim'\left( u_{a},\ u \right) \cdot \left( r_{u_{a},i} - {\overline{u}}_{a} \right)}}{\sum_{u_{a} \in S(u)}^{}{Sim'\left( u_{a},\ u \right)}}}$$

Case III:
User $u$ doesn't have similar users, but item $i$ has similar items $\Rightarrow S(u) = \varnothing \land S(i) \neq \varnothing$ :

$$\boxed{P\left( r_{u,i} \right) = \ \overline{i}\  + \frac{\sum_{i_{k} \in S(i)}^{}{Sim'\left( i_{k},\ i \right) \cdot \left( r_{u,i_{k}} - {\overline{i}}_{k} \right)}}{\sum_{i_{k} \in S(i)}^{}{Sim'\left( i_{k},\ i \right)}}}$$

Case IV:
User $u$ doesn't have similar users as well as item $i$ doesn't have similar items $\Rightarrow S(u) = \varnothing \land S(i) = \varnothing$ :

$$\boxed{P\left( r_{u,i} \right) = \ 0}$$

Here, $\lambda$ parameter (introduced in Case I) is in the range $[0,1]$ and its value determines the weighting of prediction reliability, i.e., how closely the prediction relies on user-based and item-based approach. Case II and Case III are in fact special cases of Case I where :

- $\lambda = 1 \rightarrow$ prediction depends completely on ratings from user-based approach
- $\lambda = 0 \rightarrow$ prediction depends completely on ratings from item-based approach 

### Prediction for Active Users

Following the missing data prediction, the ratings for the active users are predicted in exactly similar fashion for the first three possible cases (equations 7,8,9). But for Case IV, i.e., if user $a$ doesn't have similar users as well as item $i$ doesn't have similar items $\Rightarrow S(a) = \varnothing \land S(i) = \varnothing$, prediction is done here differently as :

$$\boxed{P\left( r_{a,i} \right) = \ \lambda \times {\overline{r}}_{a} + (1 - \lambda) \times {\overline{r}}_{i}}$$

## Implementation

### About MovieLens 100K Dataset

MovieLens data sets were collected by the [GroupLens Research Project](https://grouplens.org/) at the University of Minnesota. They operate the [MovieLens](http://www.movielens.org/) web site which is a movie recommender based on collaborative filtering, helping people find movies to watch and has hundreds of thousands of registered users. GroupLens conducts online field experiments in MovieLens in the areas of automated content recommendation, recommendation interfaces, tagging-based recommenders and interfaces, member-maintained databases, and intelligent user interface design.

Specifically, the MovieLens 100K dataset <a id="ref1" href=#1>[1]</a> was collected during the seven-month period from September 19th, 1997 through April 22nd, 1998. This rating dataset has been cleaned up - users who had less than 20 ratings or did not have complete demographic information were removed from this data set. Detailed descriptions of the data file can be found here in this [README file](https://files.grouplens.org/datasets/movielens/ml-100k-README.txt).

We will be using `u.data` dataset located inside the zipped folder for MovieLens 100K, that consists of the following :

- 100,000 ratings (1-5) by 943 users on 1682 items (movies).
- Each user has rated at least 20 movies.
- Users and items are numbered consecutively from 1. 
- The data is randomly ordered. 
- This is a tab separated list of user id | item id | rating | timestamp. 
- The time stamps are unix seconds since 1/1/1970 UTC.

### Data Import

Firstly, we import necessary `pandas` and `numpy` packages, and then load the MovieLens 100K dataset into a `pandas` dataframe.

In [1]:
import pandas as pd
import numpy as np

names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('data/ml-100k/u.data', sep='\t', names=names)
print(df.shape)
df.head()

(100000, 4)


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


The density of the user-item matrix is calculated to be $6.30\%$, which is fairly sparse.

In [2]:
n_ratings = df.shape[0]
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()
den_perc = n_ratings/(n_users*n_items)*100
print(f"Density Percentage: {den_perc:.2f}%")

Density Percentage: 6.30%


A brief summary of relevant statistics shows that the number of ratings made by users is $106.04$ on average and in the interval of $[20,737]$. Whereas, the number of ratings for items is $59.45$ on average and in the interval of $[1,583]$.

In [3]:
stats = ["max", "min","mean"]

print(f"No. of User Ratings:\n{df.pivot_table(index = ['user_id'], values = ['rating'], aggfunc = 'count').describe().loc[stats]}\n")

print(f"No. of Item Ratings:\n{df.pivot_table(index = ['item_id'], values = ['rating'], aggfunc = 'count').describe().loc[stats]}")

No. of User Ratings:
          rating
max   737.000000
min    20.000000
mean  106.044539

No. of Item Ratings:
          rating
max   583.000000
min     1.000000
mean   59.453032


We now select $500$ each of most active users and most active items from the dataset, save those in a new dataframe `df_new` to be worked with further. This new dataframe now consists of $64957$ records.

In [4]:
n_most_active_users = 500
user_ids = df.groupby('user_id').count().sort_values(by='rating', ascending=False).head(n_most_active_users).index

n_most_active_items = 500
item_ids = df.groupby('item_id').count().sort_values(by='rating', ascending=False).head(n_most_active_items).index

df_new = df[(df['user_id'].isin(user_ids)) & (df['item_id'].isin(item_ids))]
print(df_new.shape)
df_new.head()

(64957, 4)


,user_id,item_id,rating,timestamp
1,186,302,3,891717742
3,244,51,2,880606923
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467


Next, we map the internal IDs for the items to new IDs in serial order, by creating a dictionary.

In [5]:
i_ids = df_new['item_id'].unique().tolist()

item_dict = dict(zip(i_ids, [i for i in range(len(i_ids))]))

df_new = df_new.copy()
df_new['item_id'] = df_new['item_id'].map(item_dict)
df_new.head()

,user_id,item_id,rating,timestamp
1,186,0,3,891717742
3,244,1,2,880606923
5,298,2,4,884182806
6,115,3,2,881171488
7,253,4,5,891628467


### Hold-Out Sampling

We set the number of training users and active users to be 60% and 40% of the most active users, respectively.

In [6]:
n_training_users = int(0.6*n_most_active_users)
n_active_users = int(0.4*n_most_active_users)
print(f"No. of training users: {n_training_users}, No. of active users: {n_active_users}")

No. of training users: 300, No. of active users: 200


Next, we set the number of given ratings for active users to $20$  as per the data description.

In [7]:
GIVEN = 20

We randomly select users from the most active users as the training set `train_df`.

In [8]:
random_uids = np.random.choice(df.user_id.unique(), 
                               n_training_users, 
                               replace=False)

train_df = df[df['user_id'].isin(random_uids)]
train_df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
2,22,377,1,878887116
3,244,51,2,880606923
7,253,465,5,891628467
10,62,257,2,879372434


We then map the new internal IDs for all users in the training set.

In [9]:
u_ids = train_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
train_df = train_df.copy()
train_df['user_id'] = train_df['user_id'].map(user_dict)
train_df.head()

,user_id,item_id,rating,timestamp
0,0,242,3,881250949
2,1,377,1,878887116
3,2,51,2,880606923
7,3,465,5,891628467
10,4,257,2,879372434


The rest of users are active users for testing, which are assigned to `remain_df` dataframe.

In [10]:
remain_df = df[~df['user_id'].isin(random_uids)]
remain_df.head()

,user_id,item_id,rating,timestamp
1,186,302,3,891717742
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
8,305,451,3,886324817


We now map the new internal IDs for all these active users.

In [11]:
u_ids = remain_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
remain_df = remain_df.copy()
remain_df['user_id'] = remain_df['user_id'].map(user_dict)
remain_df.head()

,user_id,item_id,rating,timestamp
1,0,302,3,891717742
4,1,346,1,886397596
5,2,474,4,884182806
6,3,265,2,881171488
8,4,451,3,886324817


From these active users, we then randomly select `GIVEN` 20 ratings for the active users and assign them to `active_df` dataframe.

In [12]:
active_df = remain_df.groupby('user_id').sample(n=GIVEN, random_state=1024)
active_df.head()

,user_id,item_id,rating,timestamp
14473,0,100,4,879023115
1294,0,263,3,879023571
56711,0,1033,3,879024212
2309,0,281,4,879023390
11138,0,288,1,879022858


From the `remain_df` dataframe, we then select rest of the active users and assign them to `test_df` dataframe.

In [13]:
test_df = remain_df[~remain_df.index.isin(active_df.index)]
test_df.head()

,user_id,item_id,rating,timestamp
5,2,474,4,884182806
6,3,265,2,881171488
8,4,451,3,886324817
9,5,86,3,883603013
12,6,222,5,876042340


### Matrix Conversion

Next, we convert the format of the training dataset to matrices. 

We do this by left-merging a dataframe `df_zeros_train` that contains all combinations of $300$ training user IDs ($0$ to $299$) and all $500$ item IDs ($0$ to $499$) with the `train_df` dataframe, joined on the columns containing user IDs and item IDs. Also we fill any resulting `NA` values with zeros. Finally, we convert it to a pivot table having the user IDs as rows and item IDs as columns. This becomes the training matrix.

In [14]:
df_zeros_train = pd.DataFrame({'user_id': np.tile(np.arange(0, n_training_users),
                                                  n_most_active_items), 
                               'item_id': np.repeat(np.arange(0, n_most_active_items),
                                                    n_training_users), 
                               'rating': 0})

train_ds = df_zeros_train.merge(train_df,
                                how = 'left',
                                on = ['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', 
                                                                                    index='user_id', 
                                                                                    columns='item_id')

print(f"Training Matrix:\n{train_ds}")

Training Matrix:
item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
user_id                                                    ...                  
0        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0  0.0  ...  0.0  0.0  0.0   
1        0.0  0.0  2.0  0.0  5.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2        0.0  4.0  0.0  5.0  0.0  0.0  0.0  4.0  0.0  5.0  ...  0.0  0.0  0.0   
3        0.0  5.0  0.0  0.0  4.0  0.0  0.0  0.0  4.0  0.0  ...  5.0  0.0  0.0   
4        0.0  2.0  0.0  3.0  4.0  0.0  0.0  4.0  5.0  4.0  ...  0.0  0.0  0.0   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
295      0.0  2.0  4.0  0.0  5.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  4.0   
296      0.0  4.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0  3.0  ...  0.0  0.0  0.0   
297      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
298      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0  ...  0.0  0.0  0.0   
299      0.

In a similar way, we convert the `active_df` and `test_df` datasets to matrices.

In [15]:
df_zeros_test = pd.DataFrame({'user_id': np.tile(np.arange(0, n_active_users), 
                                                 n_most_active_items), 
                              'item_id': np.repeat(np.arange(0, n_most_active_items), 
                                                   n_active_users), 
                              'rating': 0})

active_ds = df_zeros_test.merge(active_df, 
                                how = 'left', 
                                on = ['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', 
                                                                                    index='user_id', 
                                                                                    columns='item_id')
test_ds = df_zeros_test.merge(test_df, 
                              how = 'left', 
                              on = ['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', 
                                                                                  index='user_id', 
                                                                                  columns='item_id')

print(f"Active Users Matrix:\n{active_ds}\n")
print(f"\nTesting Matrix:\n{test_ds}")

Active Users Matrix:
item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
user_id                                                    ...                  
0        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
195      0.0  4.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  ...  0.0  0.0  0.0   
196      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
197      0.0  5.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
198      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
199    

Next, we convert the training matrix `train_ds` to a `NumPy` array. This is the ___User-Item Rating Matrix___ and the missing values are represented by the zero values.

In [16]:
imputed_train_ds = train_ds.values.copy()
type(imputed_train_ds)

numpy.ndarray

### Similarity Matrix Calculation

As per the given proposition, the following parameters are required :

In [17]:
LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ETA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9  # ε

We get the number of users (rows) and items (columns) in the user-item matrix.

In [18]:
n_users = imputed_train_ds.shape[0]
n_items = imputed_train_ds.shape[1]
n_users, n_items

(300, 500)

Now, first we calculate the ___User-User Similarity Matrix___ where each cell contains the similarity of ratings for every pair of users in the training dataset. Here, we make use of masks for filtering only the ratings co-rated by current pair of users $u$ and $u_a$. No co-rated ratings are skipped here. A small value of $\text{EPSILON}\ (\epsilon)=10^{-9}$ is added to fraction denominators to prevent any undefined value occurring due to denominator being $0$. From the calculated average ratings of user pairs, we then calculate the user-user similarities(correlations) using Pearson Correlation Coefficients and apply significance weighting of $\text{GAMMA}\ (\gamma)=10$. These values are then save to a new matrix `user_corr`, the row and column indexes of which denote the user IDs.

In [19]:
user_corr = np.zeros((n_users, n_users)) 

for x, u in enumerate(imputed_train_ds):
    for y, ua in enumerate(imputed_train_ds):
        
        # ratings co-rated by current pair of users
        mask_x = u > 0
        mask_y = ua > 0
        
        # co-rated item index
        cor_index = np.intersect1d(np.where(mask_x), np.where(mask_y))
        
        # skip for no co-rated ratings
        if len(cor_index) == 0:
            continue
        
        # average ratings of current user pair
        u_mean = np.sum(u)/(np.sum(np.clip(u, 0, 1)) + EPSILON)
        ua_mean = np.sum(ua)/(np.sum(np.clip(ua, 0, 1)) + EPSILON)
        
        # user similarity using pearson correlation coefficient
        u_sum_sqrt = np.sqrt(np.sum(np.square(u[cor_index] - u_mean)))
        ua_sum_sqrt = np.sqrt(np.sum(np.square(ua[cor_index] - ua_mean)))
        sim = np.sum((u[cor_index] - u_mean) * (ua[cor_index] - ua_mean)) / (u_sum_sqrt * ua_sum_sqrt + EPSILON)
        
        # significance-weighted user similarity
        weighted_sim = (min(len(cor_index), GAMMA) / GAMMA) * sim
        
        # impute value into matrix 
        user_corr[x][y] = weighted_sim
        
print("User-User Similarities :\n", pd.DataFrame(user_corr))

User-User Similarities :
           0         1         2         3         4         5         6    \
0    1.000000  0.293778  0.145513 -0.313041  0.233893 -0.018490  0.598709   
1    0.293778  1.000000  0.224561  0.126497  0.414504  0.257308  0.276510   
2    0.145513  0.224561  1.000000 -0.198550  0.300494  0.066397  0.064681   
3   -0.313041  0.126497 -0.198550  1.000000  0.092775 -0.013063  0.130789   
4    0.233893  0.414504  0.300494  0.092775  1.000000  0.061269  0.098989   
..        ...       ...       ...       ...       ...       ...       ...   
295  0.028211  0.094499  0.127791 -0.269807  0.210176 -0.141170  0.149057   
296 -0.186404  0.628405  0.173766  0.069943  0.125097  0.038426  0.345880   
297  0.100000  0.400256  0.044220  0.172087  0.025602 -0.146467  0.244937   
298  0.270739  0.378360  0.347701  0.043618  0.653566 -0.192064  0.283573   
299 -0.369539 -0.050821  0.122013  0.632703 -0.001894  0.105897 -0.082844   

          7         8         9    ...       290 

Secondly, we calculate the ___Item-Item Similarity Matrix___ in a similar way, where each cell contains the similarity of ratings for every pair of items. Here, the row and column indexes denote the item IDs.

In [20]:
item_corr = np.zeros((n_items, n_items))

for x, i in enumerate(imputed_train_ds.T):
    for y, ik in enumerate(imputed_train_ds.T):
        
        # ratings co-rated by current pair of items
        mask_x = i > 0
        mask_y = ik > 0
        
        # co-rated user index
        cor_index = np.intersect1d(np.where(mask_x), np.where(mask_y))
        
        # skip for no co-rated ratings
        if len(cor_index) == 0:
            continue
            
        # average ratings of current item pair
        i_mean = np.sum(i)/(np.sum(np.clip(i, 0, 1)) + EPSILON)
        ik_mean = np.sum(ik)/(np.sum(np.clip(ik, 0, 1)) + EPSILON)
        
        # item similarity using pearson correlation coefficient
        i_sum_sqrt = np.sqrt(np.sum(np.square(i[cor_index] - i_mean)))
        ik_sum_sqrt = np.sqrt(np.sum(np.square(ik[cor_index] - ik_mean)))
        sim = np.sum((i[cor_index] - i_mean) * (ik[cor_index] - ik_mean)) / (i_sum_sqrt * ik_sum_sqrt + EPSILON)

        # significance-weighted item similarity
        weighted_sim = (min(len(cor_index), DELTA) / DELTA) * sim
        
        # impute value into matrix
        item_corr[x][y] = weighted_sim
        
print("Item-Item Similarities :\n", pd.DataFrame(item_corr))

Item-Item Similarities :
      0         1         2         3         4         5         6    \
0    0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1    0.0  1.000000  0.348008  0.217095 -0.032689  0.498341  0.060402   
2    0.0  0.348008  1.000000  0.016143  0.313480 -0.256172  0.100000   
3    0.0  0.217095  0.016143  1.000000 -0.256399  0.461854  0.000000   
4    0.0 -0.032689  0.313480 -0.256399  1.000000 -0.607518  0.190952   
..   ...       ...       ...       ...       ...       ...       ...   
495  0.0  0.447878  0.431988  0.200000 -0.134365  0.000000  0.100000   
496  0.0  0.418709  0.420232 -0.337227  0.172359 -0.067145 -0.100000   
497  0.0 -0.187471 -0.232230 -0.146054  0.255970 -0.200000 -0.100000   
498  0.0  0.350564 -0.070467 -0.085492 -0.208899  0.100000  0.194028   
499  0.0  0.185521 -0.106928  0.173623  0.051900 -0.088880 -0.100000   

          7         8         9    ...       490       491       492  \
0    0.000000  0.000000  0.000000  ..

### Predict Missing Values

Next, we predict all missing data in the training matrix `imputed_train_ds` by using the proposed solution framework. 

Here, we iterate through each cell of the training matrix with missing rating to generate similarity sets of users and items selected on the basis of threshold values $\text{ETA}\ (\eta)=0.7$ and $\text{THETA}\ (\theta)=0.7$ respectively. After that, we follow the formula discussed in the above sections and perform predictions based on similar user and similar item sets.

Using the previously constructed similarity matrices : `user_corr` and `item_corr`, we predict the missing data for 4 possible combinations of user and item sets, as discussed above in [section 4.5](#Missing-Data-Prediction). For missing values having null similar user and item set, we set them to zero which actually permits selective missing data prediction.

In [21]:
for (u, i), rating in np.ndenumerate(imputed_train_ds):
    if (rating == 0):
                 
        # index set of similar users for current user excluding itself, based on ETA threshold
        idx, = np.where(user_corr[u] > ETA)
        sim_user_ids = idx[np.argsort(user_corr[u][idx])[:-1]]
        
        # index set of similar items for current item excluding itself, based on THETA threshold
        idy, = np.where(item_corr[i] > THETA)
        sim_item_ids = idy[np.argsort(item_corr[i][idy])[:-1]]
                                    
        # similarity set of users
        sim_user_val = user_corr[u][sim_user_ids]
        
        # similarity set of items
        sim_item_val = item_corr[i][sim_item_ids]
      
        # average of current user's ratings
        user_mean = np.sum(imputed_train_ds[u]) / (np.sum(np.clip(imputed_train_ds[u], 0, 1)) + EPSILON)
        sim_users = imputed_train_ds[sim_user_ids]
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)                   
        
        # average of current item's ratings
        item_mean = np.sum(imputed_train_ds.T[i]) / (np.sum(np.clip(imputed_train_ds.T[i], 0, 1)) + EPSILON)
        sim_items = imputed_train_ds.T[sim_item_ids]
        sim_item_mean = np.sum(sim_items, axis=1) / (np.sum(np.clip(sim_items, 0, 1), axis=1) + EPSILON)                               
                                
        # select users who rated item i
        mask_rated_i = sim_users[:, i] > 0
        
        # sim'(ua, u) * (r_ua,i - mean_ua)
        sim_user_sum_mean = sim_user_val[mask_rated_i] * (sim_users[mask_rated_i, i] - sim_user_mean[mask_rated_i]) 
       
        # sim'(ik, i) * (r_u,ik - mean_ik)
        sim_item_sum_mean = sim_item_val * (sim_items[:, u] - sim_item_mean)
        
        # filter unrated items
        w = np.clip(sim_items[:, u], 0, 1)
        sim_item_sum_mean *= w
        
        # prediction based on similar user and similar item sets
        if (sim_user_val.size and sim_item_val.size):
            pred = LAMBDA*( user_mean + np.sum(sim_user_sum_mean) / (np.sum(sim_user_val[mask_rated_i]) + EPSILON) ) + \
                       (1-LAMBDA)*( item_mean + np.sum(sim_item_sum_mean) / (np.sum(sim_item_val * w) + EPSILON) )
        
        elif (sim_user_val.size and not sim_item_val.size):
            pred = user_mean + np.sum(sim_user_sum_mean) / (np.sum(sim_user_val[mask_rated_i]) + EPSILON)
        
        elif (not sim_user_val.size and sim_item_val.size):
            pred = item_mean + np.sum(sim_item_sum_mean) / (np.sum(sim_item_val * w) + EPSILON)
        
        elif (not sim_user_val.size and not sim_item_val.size):
            pred = 0
            
        # impute predicted value to user-item matrix
        imputed_train_ds[u][i] = np.clip(pred, 0, 5)
    else:
        continue

## Evaluation

For evaluating the predictions,  we first convert the `imputed_train_ds` which is in `NumPy` array format back to `Pandas` dataframe.

In [22]:
imputed_train_ds = pd.DataFrame(imputed_train_ds)
type(imputed_train_ds)

pandas.core.frame.DataFrame

### PCC Calculation

Next, we compute Pearson Correlation Coefficients of all pairs of items between active set and imputed training set. A similar approach as we did before for calculating the similarity matrices is followed here.

In [23]:
active_user_pearson_corr = np.zeros((active_ds.shape[0], train_ds.shape[0]))

for i, user_i_vec in enumerate(active_ds.values):
    for j, user_j_vec in enumerate(imputed_train_ds.values):
        
        # ratings corated by the current pair of users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute PCC
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        active_user_pearson_corr[i][j] = weighted_sim

active_user_pearson_corr

array([[-0.07337825, -0.00273499, -0.09900332, ...,  0.35929164,
         0.14237004,  0.13589609],
       [-0.32552554,  0.2375235 ,  0.03698918, ..., -0.14601107,
         0.05969067,  0.55183409],
       [ 0.07820159, -0.04963948,  0.11441587, ...,  0.3331637 ,
         0.28004406,  0.12890423],
       ...,
       [ 0.23214672,  0.48017719,  0.08289009, ...,  0.50221074,
         0.32219139,  0.12699871],
       [-0.24529062, -0.06940567,  0.17174559, ..., -0.16760629,
        -0.09697383,  0.27494276],
       [ 0.1591988 , -0.19660292, -0.035975  , ...,  0.09943822,
        -0.23944152, -0.23964303]])

### Prediction on Test Set

Now, we predict the ratings on the test set. For this, we iterate through the testing matrix `test_ds` for all ratings greater than zero. Then, we the find mean active user ratings, mean similar user ratings, and coefficients of similar users (using the PCCs calculated in previous section). We then proceed to calculate the user-based predictions using equation no. 8 in [section 4.5](#Missing-Data-Prediction) : 

$P\left( r_{u,i} \right) = \ \overline{u}\  + \frac{\sum_{u_{a} \in S(u)}^{}{Sim'\left( u_{a},\ u \right) \cdot \left( r_{u_{a},i} - {\overline{u}}_{a} \right)}}{\sum_{u_{a} \in S(u)}^{}{Sim'\left( u_{a},\ u \right)}}$.

Finally, we clip or limit these predicted values to interval $[1,5]$ since there are only $1$ to $5$ ratings available as per the data description.

In [24]:
K = 10

test_ds_pred = np.zeros_like(test_ds.values)

for (i, j), rating in np.ndenumerate(test_ds.values):

    if rating > 0:

        sim_user_ids = np.argsort(active_user_pearson_corr[i])[-1:-(K + 1):-1]

        #==================user-based==================#
        
        # average value of active user's ratings
        user_mean = np.sum(active_ds.values[i]) / (np.sum(np.clip(active_ds.values[i], 0, 1)) + EPSILON)   
        
        # similar user's ratings
        sim_users = imputed_train_ds.values[sim_user_ids]
        
        # average value of similar user's ratings
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)
        
        
        # coefficient values of similar users
        sim_val = active_user_pearson_corr[i][sim_user_ids]
        
        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
       
        # prediction = mean(u) + sum[sim(v,u) * (r_{v,j} - mean(v))] / sum[sim(v,u)]        
        user_based_pred = user_mean + \
                          np.sum(sim_val[mask_rated_j]*(sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])) / \
                          (np.sum(sim_val[mask_rated_j]) + EPSILON)
        
        user_based_pred = np.clip(user_based_pred, 1, 5)

        test_ds_pred[i][j] = user_based_pred
        
test_ds_pred

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 4.11197723, 0.        , ..., 0.        , 4.22378082,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### MAE and RMSE Computation

In the final step, we calculate the Mean Absolute Error (MAE) and Root Mean Square Error (RMSE) values to compare the predictions on the test set with the actual values.

In [25]:
# MAE
MAE = np.sum(np.abs(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1))

# RMSE
RMSE = np.sqrt(np.sum(np.square(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1)))

print(f"MAE: {MAE}\nRMSE: {RMSE}")

MAE: 0.7956406973413431
RMSE: 1.0200527938518875


## Impact of EMDP

- The algorithm discussed incorporates the option of not predicting missing data if it doesn't meet the criteria set using threshold values. 

- Further, it removes potential negative influences from inaccurate prediction on the missing data. 

- The effectiveness of this approach is investigated in the given paper using simulations on a dataset and plotting Mean Absolute Error vs. $\lambda$.

- Mean Absolute Error (MAE) metrics is used here to measure the prediction quality of our proposed approach with other collaborative filtering methods, which is given by:

$$\text{MAE}=\frac{\sum_{u,i}\left | r_{u,i}-\hat{r}_{u,i} \right |}{N}$$
where<br>
$r_{u,i} \rightarrow$ actual rating that user $u$ gave to item $i$,<br>
$\hat{r}_{u,i} \rightarrow$ predicted rating that user $u$ gave to item $i$,<br>
$N \rightarrow$ number of tested ratings.

<p style="text-align:center;">
    <img src="images/mae_comparison.png" alt="mae_comparison" title="MAE Comparison" width="400"><br><center><i>Fig. 7: MAE Comparison of EMDP vs PEMD <a id="ref3" href=#3>[3]</a></i></center>
</p>

Consequently, this proposed method's (EMDP) performance is compared with that of an approach which predicts every missing data (PEMD). The results show that this selective missing-data prediction algorithm has lower MAE value than the one that predicts every missing throughout the range of $\lambda$. Therefore, this algorithm performs better and selectively predicting missing data is more effective.

## Ending Notes

- We implemented an Effective Missing Data Prediction (EMDP) algorithm for Collaborative Filtering as per the solution framework provided in the given paper. 

- This algorithm performs on combined information from and user and item similarities.

- It selectively predicts missing data. 

- The algorithm outperformed other missing data prediction approaches, so it is effective for use in Collaborative Filtering.

***

## References




<a id="1" href=#ref1>[1]</a> Harper, F.M. and Konstan, J.A. (2015). The MovieLens Datasets: History and Context. _ACM Transactions on Interactive Intelligent Systems (TiiS)_, [online] 5(4), pp.1–19, Article No.: 19. doi:10.1145/2827872.

<a id="2" href=#ref2>[2]</a> Koren, Y., Bell, R. and Volinsky, C. (2009). Matrix Factorization Techniques for Recommender Systems. _Computer_, [online] 42(8), pp.30–37. doi:10.1109/mc.2009.263.

<a id="3" href=#ref3>[3]</a> Ma, H., King, I. and Lyu, M.R. (2007). Effective missing data prediction for collaborative filtering. In: _Proceedings of the 30th annual international ACM SIGIR conference on Research and development in information retrieval - SIGIR ’07_. [online] Amsterdam, The Netherlands: Association for Computing Machinery, pp.39–46. doi:10.1145/1277741.1277751.

<a id="4" href=#ref4>[4]</a> McDonald, C. and Moreira, G. (2021). _How to Build a Winning Recommendation System, Part 1_. [online] NVIDIA Developer Blog. Available at: https://developer.nvidia.com/blog/how-to-build-a-winning-recommendation-system-part-1/ [Accessed 11 Jun. 2021].

<a id="5" href=#ref5>[5]</a> Nixon, À.E. (2021). _Building a Memory Based Collaborative Filtering Recommender_. [online] Towards Data Science - Medium. Available at: https://towardsdatascience.com/how-does-collaborative-filtering-work-da56ea94e331 [Accessed 11 Jun. 2019].

<a id="6" href=#ref6>[6]</a> Parks, J., Ramm, M. and Aurisset, J. (2020). _Innovating Faster on Personalization Algorithms at Netflix Using Interleaving_. [online] Netflix Technology Blog - Medium. Available at: https://netflixtechblog.com/interleaving-in-online-experiments-at-netflix-a04ee392ec55 [Accessed 13 Jun. 2021].

<a id="7" href=#ref7>[7]</a> Parveez, S. and Iriondo, R. (2021). _Recommendation System Tutorial with Python Using Collaborative Filtering_. [online] Towards AI - Medium. Available at: https://pub.towardsai.net/recommendation-system-in-depth-tutorial-with-python-for-netflix-using-collaborative-filtering-533ff8a0e444 [Accessed 10 Jun. 2021].

<a id="8" href=#ref8>[8]</a> Sarwar, B., Karypis, G., Konstan, J. and Reidl, J. (2001). Item-Based Collaborative Filtering Recommendation Algorithms. _Proceedings of the Tenth International Conference on World Wide Web - WWW ’01_, [online] pp.285–295. doi:10.1145/371920.372071.